# Task 1.2 Data Analysis Project City Bike NYC

## Goals
This dataset contains a sample of bike trips from the Citi Bike system in New York City.
Each row represents one trip and includes information about the start and end stations, the duration, the
user type, and other contextual data like age, season, temperature, and weekday.
Your goal is to explore this dataset and extract insights through data analysis with pandas.

You'll practice basic pandas operations (loading, exploring, cleaning, transforming, summarizing) and use descriptive statistics and simple visualizations to support your answers.

In [80]:
import pandas as pd

pd.read_excel("ny_citibikes_raw.xlsx").to_csv("ny_citibikes_raw.csv", index=False)
df = pd.read_csv("ny_citibikes_raw.csv")
df.head()


,Start Time,Stop Time,Start Station ID,Start Station Name,End Station ID,End Station Name,Bike ID,User Type,Birth Year,Age,Age Groups,Trip Duration,Trip_Duration_in_min,Month,Season,Temperature,Weekday
0,2017-01-01 00:38:00,2017-01-01 01:03:00,3194,McGinley Square,3271,Danforth Light Rail,24668,Subscriber,1961,60,55-64,1513,25,1,Winter,10,Sunday
1,2017-01-01 01:47:00,2017-01-01 01:58:00,3183,Exchange Place,3203,Hamilton Park,26167,Subscriber,1993,28,25-34,639,11,1,Winter,10,Sunday
2,2017-01-01 01:47:00,2017-01-01 01:58:00,3183,Exchange Place,3203,Hamilton Park,26167,Subscriber,1993,28,25-34,639,11,1,Winter,10,Sunday
3,2017-01-01 01:56:00,2017-01-01 02:00:00,3186,Grove St PATH,3270,Jersey & 6th St,24604,Subscriber,1970,51,45-54,258,4,1,Winter,10,Sunday
4,2017-01-01 02:12:00,2017-01-01 02:23:00,3270,Jersey & 6th St,3206,Hilltop,24641,Subscriber,1978,43,35-44,663,11,1,Winter,10,Sunday


## 1. Dataset Exploration
- What information does each column contain?

Con los comandos df.head() y df.info() nos podemos hacer una idea de los contenidos de cada columna. Viendo los resultados se puede hacer un resumen más "humanizado" de cada columna:

- **Start Time**: guarda la fecha y hora de inicio del viaje como Object.
- **Stop Time**: guarda la fecha y hora de finalizacion del viaje como  Object.
- **Start Station ID**: guarda el identificador de la estación de comienzo del viaje como Int64.
- **Start Station Name**: guarda el nombre de la estación de comienzo del viaje como Object.
- **End Station ID**: guarda el identificador de la estación de finalización del viaje como Int64.
- **End Station Name**: guarda el nombre de la estación de finalización del viaje como Object.
- **Bike ID**: guarda el identificador de la bicicleta usada en el viaje como Int64.
- **User Type**: guarda el tipo de usuario que realiza el viaje como Object.
- **Birth Year**: guarda el año de nacimiento del usuario que realiza el viaje como Int64.
- **Age**: guarda la edad del usuario que realiza el viaje como Int64.
- **Age Groups**: guarda el intervalo de edad al que pertenece el usuario que realiza el viaje como Object.
- **Trip Duration**: guarda la duración del viaje en segundos como Int64.
- **Trip_Duration_in_min**: guarda la duración del viaje en minutos como Int64.
- **Month**: guarda el número que corresponde al mes en el que se realiza el viaje como Int64.
- **Season**: guarda la estación del año en la que se realiza el viaje como Object.
- **Temperature**: guarda la temperatura en grados celsius del momento en el que se realiza el viaje como Int64.
- **Weekday**: guarda el día de la semana en el que se realiza el viaje como Object.

Con esta información podemos hacernos una idea de cómo sacar los datos para analizarlos y que datos hay que limpiar u obviar. Si queda alguna duda siempre podemos indagar un poco más en la columna que no se entienda del todo. Por ejemplo quiero saber que datos se guardan en user type, porque a primera vista parece que se podría manejar con una columna "Subscriber" con resultados 0 y 1. Con un value_counts() puedo visualizar los datos que guarda, y con ello puedo sacar la conclusión de que si que se podría manejar con un booleano o un int que maneje 0 o 1.


In [81]:
df.info()
df["User Type"].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20400 entries, 0 to 20399
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Start Time            20400 non-null  object
 1   Stop Time             20400 non-null  object
 2   Start Station ID      20400 non-null  int64 
 3   Start Station Name    20400 non-null  object
 4   End Station ID        20400 non-null  int64 
 5   End Station Name      20399 non-null  object
 6   Bike ID               20400 non-null  int64 
 7   User Type             20400 non-null  object
 8   Birth Year            20400 non-null  int64 
 9   Age                   20400 non-null  int64 
 10  Age Groups            20400 non-null  object
 11  Trip Duration         20400 non-null  int64 
 12  Trip_Duration_in_min  20400 non-null  int64 
 13  Month                 20400 non-null  int64 
 14  Season                20400 non-null  object
 15  Temperature           20400 non-null

User Type
Subscriber       20020
One-time user      380
Name: count, dtype: int64

- Are there missing or duplicated values?

Si, hay un total de 3555 filas duplicadas en el dataset, estos registros se deben eliminar para no alterar las estadísticas con un volumen de datos que no es realista.

 Después de comprobar los duplicados se pueden comprobar los nulos que hay por columna. En este caso nos da un nulo en un nombre de la estación de llegada. Como no nos da el mismo número en el ID de la estación, podemos buscar si el ID coincide con alguna otra estación que se haya registrado.

In [82]:
print(f"Número de duplicados antes de la eliminación: {df.duplicated().sum()}\n")
df = df.drop_duplicates()
print(f"Número de duplicados después de la eliminación: {df.duplicated().sum()}\n")

# Comprobamos los nulos por columna:
print(f"Número de nulos por columna:\n{df.isna().sum()}\n")

# Buscamos si el nulo en el nombre de la estación tiene un ID repetido en alguna otra entrada.
print(df[df['End Station Name'].isna()])
print(df[df['End Station ID'] == 3211])

# Una vez comprobado podemos rellenar ese nulo con el nombre de la estación asociada al id, en este caso "Newark Ave".
df.loc[9858, 'End Station Name'] = 'Newark Ave'

# Volvemos a comprobar los nulos para ver si el campo se ha cubierto correctamente.
print(f"Número de nulos por columna: {df.isna().sum()}\n")

# Mostramos el campo con el nombre de la estación cambiada.
print(df.loc[9858])

Número de duplicados antes de la eliminación: 3555

Número de duplicados después de la eliminación: 0

Número de nulos por columna:
Start Time              0
Stop Time               0
Start Station ID        0
Start Station Name      0
End Station ID          0
End Station Name        1
Bike ID                 0
User Type               0
Birth Year              0
Age                     0
Age Groups              0
Trip Duration           0
Trip_Duration_in_min    0
Month                   0
Season                  0
Temperature             0
Weekday                 0
dtype: int64

               Start Time            Stop Time  Start Station ID  \
9858  2017-02-25 12:36:00  2017-02-25 12:44:00              3220   

     Start Station Name  End Station ID End Station Name  Bike ID   User Type  \
9858  5 Corners Library            3211              NaN    24522  Subscriber   

      Birth Year  Age Age Groups  Trip Duration  Trip_Duration_in_min  Month  \
9858        1963   58      55-64

- What is the overall time span of the trips?

## 2. Basic Statistics
- What is the average trip duration (in minutes)?

La duración media de los viajes en minutos es aproximado a 9.58 mins/viaje. (Exactamente: 9.578272484416742 minutos)

In [83]:
print(f"Duración media en minutos por viaje: {df['Trip_Duration_in_min'].mean()}")

Duración media en minutos por viaje: 9.578272484416742


- What is the minimum and maximum duration?

La duración del viaje más corto es 1 minuto y la duración del viaje más largo son 6515 minutos, que son 109 horas que a su vez son 4 días y medio.

In [91]:
print(f"Duración mínima: {df['Trip_Duration_in_min'].min()}")
print(f"Duración máxima: {df['Trip_Duration_in_min'].max()}")

Duración mínima: 1
Duración máxima: 6515


- What are the most common start and end stations?

Ambos resultados pertenecen a la estación de **Grove St PATH** con un total de 4858 registros. 2115 registros como estación de salida y 2743 registros como estación de llegada.

In [103]:
print(f"\n\nEstación de salida más frecuentada: \n{df['Start Station Name'].value_counts().head(1)}")
print(f"\n\nEstación de llegada más frecuentada: \n{df['End Station Name'].value_counts().head(1)}")



Estación de salida más frecuentada: 
Start Station Name
Grove St PATH    2115
Name: count, dtype: int64


Estación de llegada más frecuentada: 
End Station Name
Grove St PATH    2743
Name: count, dtype: int64


## 3. Users and Demographics
- How many unique bikes were used?

Se han usado un total de 500 bicicletas en un total de 16845 viajes.

In [112]:
print(f"Número de bicis únicas usadas: {df['Bike ID'].nunique()}\n\n")
df.info()

Número de bicis únicas usadas: 500


<class 'pandas.core.frame.DataFrame'>
Index: 16845 entries, 0 to 20399
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Start Time            16845 non-null  object
 1   Stop Time             16845 non-null  object
 2   Start Station ID      16845 non-null  int64 
 3   Start Station Name    16845 non-null  object
 4   End Station ID        16845 non-null  int64 
 5   End Station Name      16845 non-null  object
 6   Bike ID               16845 non-null  int64 
 7   User Type             16845 non-null  object
 8   Birth Year            16845 non-null  int64 
 9   Age                   16845 non-null  int64 
 10  Age Groups            16845 non-null  object
 11  Trip Duration         16845 non-null  int64 
 12  Trip_Duration_in_min  16845 non-null  int64 
 13  Month                 16845 non-null  int64 
 14  Season                16845 non-null  object
 15  Temp

- What are the proportions of user types (Subscriber vs Customer)?

Hay un 98% de subscriptores del servicio usando las bicis, y un 1.89% de usuarios que no están suscritos al servicio.

In [114]:
usuarios = df['User Type'].value_counts()
print(usuarios / usuarios.sum() *100)

User Type
Subscriber       98.112199
One-time user     1.887801
Name: count, dtype: float64


- What is the age distribution of the users? Which age group uses the service the most?

Los usuarios que más utilizan los servicios están entre los 35 y 44 años de edad. Forman un total de 7698 usuarios pertenecientes a ese grupo de edad.

In [115]:
df['Age Groups'].value_counts()

Age Groups
35-44    7698
25-34    4002
45-54    2973
55-64    1448
65-74     615
75+        55
18-24      54
Name: count, dtype: int64

## 4. Temporal Analysis
- How does the number of trips vary by weekday?
- Which month or season has the most rides?
- What time of day do most trips start?

## 5. Geographic Analysis
- Which station pairs (start → end) appear most often?
- Are there any stations that appear only as start or only as end stations?

## 6. Temperature and Duration
- Is there any visible relationship between temperature and trip duration?
- How does average trip duration vary by season?

## 7. Summary and Interpretation
- Write a short summary (5–10 lines) of your findings.
- Mention patterns, anomalies, or interesting trends you observed.